pip install xgboost


# Multi-Level Stacking Classification

#Importing The libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
from sklearn.linear_model import RidgeClassifierCV
from sklearn.svm import LinearSVC
from sklearn import linear_model
from sklearn.ensemble import StackingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import  LassoCV
from sklearn.svm import SVC
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import SGDClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score

In [107]:
dataset = pd.read_csv('nasa.csv')
print(dataset.columns)

Index(['Neo Reference ID', 'Name', 'Absolute Magnitude', 'Est Dia in KM(min)',
       'Est Dia in KM(max)', 'Est Dia in M(min)', 'Est Dia in M(max)',
       'Est Dia in Miles(min)', 'Est Dia in Miles(max)',
       'Est Dia in Feet(min)', 'Est Dia in Feet(max)', 'Close Approach Date',
       'Epoch Date Close Approach', 'Relative Velocity km per sec',
       'Relative Velocity km per hr', 'Miles per hour',
       'Miss Dist.(Astronomical)', 'Miss Dist.(lunar)',
       'Miss Dist.(kilometers)', 'Miss Dist.(miles)', 'Orbiting Body',
       'Orbit ID', 'Orbit Determination Date', 'Orbit Uncertainity',
       'Minimum Orbit Intersection', 'Jupiter Tisserand Invariant',
       'Epoch Osculation', 'Eccentricity', 'Semi Major Axis', 'Inclination',
       'Asc Node Longitude', 'Orbital Period', 'Perihelion Distance',
       'Perihelion Arg', 'Aphelion Dist', 'Perihelion Time', 'Mean Anomaly',
       'Mean Motion', 'Equinox', 'Hazardous'],
      dtype='object')


In [108]:
dataset.head()

,Neo Reference ID,Name,Absolute Magnitude,Est Dia in KM(min),Est Dia in KM(max),Est Dia in M(min),Est Dia in M(max),Est Dia in Miles(min),Est Dia in Miles(max),Est Dia in Feet(min),...,Asc Node Longitude,Orbital Period,Perihelion Distance,Perihelion Arg,Aphelion Dist,Perihelion Time,Mean Anomaly,Mean Motion,Equinox,Hazardous
0,3703080,3703080,21.6,0.127220,0.284472,127.219879,284.472297,0.079051,0.176763,417.388066,...,314.373913,609.599786,0.808259,57.257470,2.005764,2.458162e+06,264.837533,0.590551,J2000,True
1,3723955,3723955,21.3,0.146068,0.326618,146.067964,326.617897,0.090762,0.202951,479.225620,...,136.717242,425.869294,0.718200,313.091975,1.497352,2.457795e+06,173.741112,0.845330,J2000,False
2,2446862,2446862,20.3,0.231502,0.517654,231.502122,517.654482,0.143849,0.321655,759.521423,...,259.475979,643.580228,0.950791,248.415038,1.966857,2.458120e+06,292.893654,0.559371,J2000,True
3,3092506,3092506,27.4,0.008801,0.019681,8.801465,19.680675,0.005469,0.012229,28.876199,...,57.173266,514.082140,0.983902,18.707701,1.527904,2.457902e+06,68.741007,0.700277,J2000,False
4,3514799,3514799,21.6,0.127220,0.284472,127.219879,284.472297,0.079051,0.176763,417.388066,...,84.629307,495.597821,0.967687,158.263596,1.483543,2.457814e+06,135.142133,0.726395,J2000,True


In [109]:
dataset['Hazardous'].unique()

array([ True, False])

In [110]:
# Converting to numerical values
from sklearn import preprocessing
num = preprocessing.LabelEncoder()
dataset.Hazardous = num.fit_transform(dataset.Hazardous)
dataset['Close Approach Date'] = num.fit_transform(dataset['Close Approach Date'])
dataset['Orbit Determination Date'] = num.fit_transform(dataset['Orbit Determination Date'])
dataset['Orbiting Body']= num.fit_transform(dataset['Orbiting Body'])
dataset['Equinox'] = num.fit_transform(dataset['Equinox'])

In [111]:
dataset['Hazardous'].unique()

array([1, 0], dtype=int64)

In [112]:
dataset.head()

,Neo Reference ID,Name,Absolute Magnitude,Est Dia in KM(min),Est Dia in KM(max),Est Dia in M(min),Est Dia in M(max),Est Dia in Miles(min),Est Dia in Miles(max),Est Dia in Feet(min),...,Asc Node Longitude,Orbital Period,Perihelion Distance,Perihelion Arg,Aphelion Dist,Perihelion Time,Mean Anomaly,Mean Motion,Equinox,Hazardous
0,3703080,3703080,21.6,0.127220,0.284472,127.219879,284.472297,0.079051,0.176763,417.388066,...,314.373913,609.599786,0.808259,57.257470,2.005764,2.458162e+06,264.837533,0.590551,0,1
1,3723955,3723955,21.3,0.146068,0.326618,146.067964,326.617897,0.090762,0.202951,479.225620,...,136.717242,425.869294,0.718200,313.091975,1.497352,2.457795e+06,173.741112,0.845330,0,0
2,2446862,2446862,20.3,0.231502,0.517654,231.502122,517.654482,0.143849,0.321655,759.521423,...,259.475979,643.580228,0.950791,248.415038,1.966857,2.458120e+06,292.893654,0.559371,0,1
3,3092506,3092506,27.4,0.008801,0.019681,8.801465,19.680675,0.005469,0.012229,28.876199,...,57.173266,514.082140,0.983902,18.707701,1.527904,2.457902e+06,68.741007,0.700277,0,0
4,3514799,3514799,21.6,0.127220,0.284472,127.219879,284.472297,0.079051,0.176763,417.388066,...,84.629307,495.597821,0.967687,158.263596,1.483543,2.457814e+06,135.142133,0.726395,0,1


In [113]:
X = dataset.drop('Hazardous',axis=1)
X

,Neo Reference ID,Name,Absolute Magnitude,Est Dia in KM(min),Est Dia in KM(max),Est Dia in M(min),Est Dia in M(max),Est Dia in Miles(min),Est Dia in Miles(max),Est Dia in Feet(min),...,Inclination,Asc Node Longitude,Orbital Period,Perihelion Distance,Perihelion Arg,Aphelion Dist,Perihelion Time,Mean Anomaly,Mean Motion,Equinox
0,3703080,3703080,21.600,0.127220,0.284472,127.219879,284.472297,0.079051,0.176763,417.388066,...,6.025981,314.373913,609.599786,0.808259,57.257470,2.005764,2.458162e+06,264.837533,0.590551,0
1,3723955,3723955,21.300,0.146068,0.326618,146.067964,326.617897,0.090762,0.202951,479.225620,...,28.412996,136.717242,425.869294,0.718200,313.091975,1.497352,2.457795e+06,173.741112,0.845330,0
2,2446862,2446862,20.300,0.231502,0.517654,231.502122,517.654482,0.143849,0.321655,759.521423,...,4.237961,259.475979,643.580228,0.950791,248.415038,1.966857,2.458120e+06,292.893654,0.559371,0
3,3092506,3092506,27.400,0.008801,0.019681,8.801465,19.680675,0.005469,0.012229,28.876199,...,7.905894,57.173266,514.082140,0.983902,18.707701,1.527904,2.457902e+06,68.741007,0.700277,0
4,3514799,3514799,21.600,0.127220,0.284472,127.219879,284.472297,0.079051,0.176763,417.388066,...,16.793382,84.629307,495.597821,0.967687,158.263596,1.483543,2.457814e+06,135.142133,0.726395,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4682,3759007,3759007,23.900,0.044112,0.098637,44.111820,98.637028,0.027410,0.061290,144.723824,...,39.880491,164.183305,457.179984,0.741558,276.395697,1.581299,2.457708e+06,304.306025,0.787436,0
4683,3759295,3759295,28.200,0.006089,0.013616,6.089126,13.615700,0.003784,0.008460,19.977449,...,5.360249,345.225230,407.185767,0.996434,42.111064,1.153835,2.458088e+06,282.978786,0.884117,0
4684,3759714,3759714,22.700,0.076658,0.171412,76.657557,171.411509,0.047633,0.106510,251.501180,...,4.405467,37.026468,690.054279,0.965760,274.692712,2.090708,2.458300e+06,203.501147,0.521698,0
4685,3759720,3759720,21.800,0.116026,0.259442,116.025908,259.441818,0.072095,0.161210,380.662441,...,21.080244,163.802910,662.048343,1.185467,180.346090,1.787733,2.458288e+06,203.524965,0.543767,0


In [114]:
y =  dataset['Hazardous']
y

0       1
1       0
2       1
3       0
4       1
       ..
4682    0
4683    0
4684    0
4685    0
4686    0
Name: Hazardous, Length: 4687, dtype: int64

In [115]:
# splitting the dataset into the training set and test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state= 0)

In [116]:
from sklearn.preprocessing import LabelEncoder

# Initialize LabelEncoder
Le = LabelEncoder()

# fit and transform the target variable
y_train_encoded = Le.fit_transform(y_train)

In [117]:
X_train

,Neo Reference ID,Name,Absolute Magnitude,Est Dia in KM(min),Est Dia in KM(max),Est Dia in M(min),Est Dia in M(max),Est Dia in Miles(min),Est Dia in Miles(max),Est Dia in Feet(min),...,Inclination,Asc Node Longitude,Orbital Period,Perihelion Distance,Perihelion Arg,Aphelion Dist,Perihelion Time,Mean Anomaly,Mean Motion,Equinox
3002,2259221,2259221,19.200,0.384198,0.859093,384.197891,859.092601,0.238729,0.533815,1260.491809,...,23.706171,308.879323,421.543460,0.183632,18.136215,2.016891,2.458049e+06,318.335491,0.854004,0
2714,3472299,3472299,20.900,0.175612,0.392681,175.612318,392.681082,0.109120,0.244001,576.155919,...,42.048735,202.096538,270.485005,0.559791,30.560633,1.077251,2.455025e+06,139.170101,1.330943,0
408,3512702,3512702,25.400,0.022108,0.049436,22.108281,49.435619,0.013737,0.030718,72.533733,...,0.639551,30.495041,448.971034,0.817850,64.780664,1.477118,2.457900e+06,80.645481,0.801833,0
2863,2163693,2163693,16.300,1.460680,3.266179,1460.679643,3266.178974,0.907624,2.029509,4792.256199,...,25.617807,103.899114,233.022027,0.502342,252.932420,0.979820,2.457948e+06,81.777020,1.544918,0
3039,3557828,3557828,25.800,0.018389,0.041119,18.388867,41.118757,0.011426,0.025550,60.330931,...,10.584329,152.138327,469.541027,1.025627,7.221443,1.338914,2.455621e+06,357.187676,0.766706,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1033,3599541,3599541,22.400,0.088015,0.196807,88.014652,196.806745,0.054690,0.122290,288.761991,...,8.775282,347.432718,336.252832,0.713700,356.964395,1.178970,2.457834e+06,177.845666,1.070623,0
3264,3554011,3554011,22.100,0.101054,0.225964,101.054342,225.964377,0.062792,0.140408,331.543126,...,4.477541,279.018854,313.609038,0.647245,334.454171,1.159472,2.457891e+06,125.894066,1.147926,0
1653,3775095,3775095,26.526,0.013163,0.029433,13.163003,29.433369,0.008179,0.018289,43.185706,...,5.913391,69.091383,600.741768,0.847130,245.877935,1.939566,2.457955e+06,27.557507,0.599259,0
2607,2085774,2085774,19.200,0.384198,0.859093,384.197891,859.092601,0.238729,0.533815,1260.491809,...,13.589367,64.693245,607.438081,0.941721,50.043012,1.865645,2.455442e+06,135.428414,0.592653,0


In [118]:
X_train.dtypes

Neo Reference ID                  int64
Name                              int64
Absolute Magnitude              float64
Est Dia in KM(min)              float64
Est Dia in KM(max)              float64
Est Dia in M(min)               float64
Est Dia in M(max)               float64
Est Dia in Miles(min)           float64
Est Dia in Miles(max)           float64
Est Dia in Feet(min)            float64
Est Dia in Feet(max)            float64
Close Approach Date               int32
Epoch Date Close Approach         int64
Relative Velocity km per sec    float64
Relative Velocity km per hr     float64
Miles per hour                  float64
Miss Dist.(Astronomical)        float64
Miss Dist.(lunar)               float64
Miss Dist.(kilometers)          float64
Miss Dist.(miles)               float64
Orbiting Body                     int32
Orbit ID                          int64
Orbit Determination Date          int32
Orbit Uncertainity                int64
Minimum Orbit Intersection      float64


In [119]:
# feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.fit_transform(X_test)
#y_train = y_train.reshape(1,-1)
#y_test = y_test.reshape(-1, 1)
#sc.fit(y_train)
#y_train = sc.transform(y_train)

In [121]:
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.metrics import  accuracy_score
from sklearn.preprocessing import LabelEncoder

# Ensure that y_train contains discrete class labels
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)

final_layer = StackingClassifier(
    estimators=[
        ('xgbc', XGBClassifier(criterion = 'gini', max_depth=20, max_features = 'auto', min_sample_leaf=0, n_estimators=10)),
        ('sgd', SGDClassifier(max_iter=3000, tol=1e-3)),
        ('dtc', DecisionTreeClassifier(random_state = 42)),
        ('adac', AdaBoostClassifier(random_state = 42, n_estimators =100)),
        ('gbt', HistGradientBoostingClassifier(random_state=42)),
        ('rfc', RandomForestClassifier(random_state=42)),
        ('etc', ExtraTreesClassifier(n_estimators=100, max_depth=5, max_features=0.5, random_state=42))],
        final_estimator=RidgeClassifierCV(), passthrough=True, n_jobs =-1)


multi_layer_classifier = StackingClassifier(
    estimators= [
        ('lr', RidgeClassifierCV()),
        ('lasso', LassoCV(random_state=42)),
        ('lrr', linear_model.LogisticRegression()),
        ('nn', MLPClassifier(hidden_layer_sizes=(250, 700, 102, 51), max_iter=3500,
activation='relu', solver = 'adam', alpha =0.01, random_state=42)),
        ],
        final_estimator = final_layer, passthrough= False, n_jobs =-1
)

multi_layer_classifier.fit(X_train, y_train_encoded)

#Perform Prediction
y_pred = multi_layer_classifier.predict(X_test)

#Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")


Accuracy: 0.9797441364605544


In [123]:
from sklearn.model_selection import cross_val_score

# perform K-fold cross validation

scores = cross_val_score(estimator=multi_layer_classifier, X=X_train, y=y_train_encoded, cv=5, scoring='accuracy')

# Print the cross_validation score
print("Cross-Validation Scores:", scores)

#Calculate the mean of the cross_validation scores
mean_accuracy = scores.mean()
print("Mean Accuracy:", mean_accuracy)


# Calcualte the standard deviation of the cross validation scores
std_deviation = scores.std() *100
print("Standard Deviation:", std_deviation)

Cross-Validation Scores: [0.98266667 0.97066667 0.97866667 0.972      0.9799733 ]
Mean Accuracy: 0.9767946595460615
Standard Deviation: 0.4661108792614286


In [130]:
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import cross_val_score

#Define the numble of samples
n_samples = X_train.shape[0]

# Define Shufflesplit cross_validation strategy
cv = ShuffleSplit(n_splits=5, test_size=0.2, random_state=0)

# perform croos-validtaton
scores = cross_val_score(multi_layer_classifier, X_train, y_train_encoded, cv=cv, scoring='accuracy', n_jobs=-1)

# print the cross validation
print("Cross Validation Scores:", scores)

# Calcaluate the mean of the cross validation scores
mean_accuracy = scores.mean()
print("Mean Accuracy:", mean_accuracy)

#Calculate the standard devaition of the cross validation score
std_deviaton =  scores.std() *100
print(" Standard Deviation:", std_deviation)

Cross Validation Scores: [0.97733333 0.972      0.97466667 0.972      0.98533333]
Mean Accuracy: 0.9762666666666666
 Standard Deviation: 0.4661108792614286


In [4]:
from sklearn.model_selection import RandomizedSearchCV
from time import time

# Define the parameter grid for classification
params = {
    'final_estimator__xgbc__max_depth':[20, 23, 25],
    'final_estimator__etc__max_features':[0.5, 1],
    'final_estimator__etc__max_depth': [5, 10, 20],
    'final_estimator__adac__n_estimators':[100, 200]}
    #'svr__gamma':['scale', 'auto', 1e-6]}

#Run Randomized search
n_iter_search =15
random_search = RandomizedSearchCV(multi_layer_classifier, param_distributions=params, cv=cv, n_iter=n_iter_search, n_jobs=-1)

start = time()
random_search.fit(X_train, y_train_encoded)

print("RandomizedSearchCV took %.2f seconds for %d candidates"
        "parameter settings." %((time() - start), n_iter_search))

NameError: name 'multi_layer_classifier' is not defined

In [3]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

#Random Search Model
# Predicting the Test set results

best_error = random_search.best_score
best_parameters = random_search.best_params
print("Best Error: %".format(best_error*100))
print("Best Parameters:", best_parameters)

# Perform Predictions
y_pred= random_search.predict(X_test)

#Calculate Accuracy
accuracy =  accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

#calculate Precision
precision = precision_score(y_test, y_pred, average='weighted')
print(f"Precision: {precision}")

#Calculate Recall
recall = recall_score(y_test, y_pred, average= 'weighted')
print(f"Precision: {precision}")

#calculate recall
recall = recall_score(y_test, y_pred, average='weighted')
print(f"Recall: {recall}")

#calcualte F1 Score
f1 =f1_score(y_test, y_pred, average='weighted')
print(f"F1 score: {f1}")

NameError: name 'random_search' is not defined